In [ ]:
using PanTaGruEl
using Impute
using DataFrames

In [ ]:
country = ["AL", "AT", "BA", "BE", "BG", "CH", "CZ", "DE", "ES",
    "FR", "GR", "HR", "HU", "IT", "LU", "ME", "MK",
    "NL", "PL", "PT", "RO", "RS", "SI", "SK", "DK"];

In [ ]:
source_folder = "../sources"

In [ ]:
df = create_entsoe_list(source_folder, country)

In [ ]:
# Turn zeros into missing to use impute to fill missing data
allowmissing!(df)
for name in names(df)
    if name == "Date"
        continue
    end
    df[iszero.(df[!, name]), name] .= missing
    df[!, name] = (
                Impute.impute(df[!, name],
                              Impute.Interpolate(; limit=12)) |> 
                (x -> Impute.impute(x, Impute.LOCF(; limit=3))) |> 
                (x -> Impute.impute(x, Impute.NOCB(; limit=3))) |> 
                (x -> Impute.impute(x, Impute.Replace(; values=0.0))))
end

In [ ]:
# Show missing data
for name in names(df)
    if name == "Date"
        continue
    end
    ct = count(iszero.(df[!,name]))
    println("$name => $ct")
end